# ２章 小売店のデータ加工

In [1]:
import pandas as pd
uriage_data = pd.read_csv("uriage.csv")
kokyaku_data = pd.read_excel("kokyaku_daicho.xlsx")

uriage_data.isnull().sum()

purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64

In [7]:
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [5]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


### NULLがあったり、データ形式が揃っていなかったりするが、一旦そのまま集計してみる
### 売上履歴から、商品毎の月売上合計を計算する

In [12]:
#日時をdatetimeに変換する
uriage_data["purchase_date"] = pd.to_datetime(uriage_data["purchase_date"])
#日付を年月の形で格納する
uriage_data["purchase_month"] = uriage_data["purchase_date"].dt.strftime("%Y%m")
#縦軸を年月、横軸を商品名としてテーブル化する
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", aggfunc="size", fill_value=0) #fill_valueは欠損値を補完する値の指定(デフォはNone)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,商 品T,商 品V,商 品 S,商 品 a,商 品 q,商 品 s,商 品A,商 品C,商 品D,商 品E,商 品F,商 品G,商 品H,商 品I,商 品K,商 品M,商 品N,商 品O,商 品P,商 品T,商 品U,商 品V,商 品X,商 品Y,商 品s,商品 A,商品 B,商品 E,商品 F,商品 H,...,商品D,商品E,商品F,商品G,商品H,商品I,商品J,商品K,商品L,商品M,商品N,商品O,商品P,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z,商品a,商品c,商品d,商品e,商品g,商品i,商品j,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,2,0,1,0,0,0,0,0,...,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
201903,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,1,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,...,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,...,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0


### 本来A~Zの26個なのに99に商品が増えてしまっている
### 同様に横軸をitem_priceにしてみる

In [13]:
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", values="item_price",aggfunc="sum",fill_value=0)
res

item_name,商品W,商 品 n,商品E,商品M,商品P,商品S,商品W,商品X,商 品O,商 品Q,商 品T,商 品V,商 品 S,商 品 a,商 品 q,商 品 s,商 品A,商 品C,商 品D,商 品E,商 品F,商 品G,商 品H,商 品I,商 品K,商 品M,商 品N,商 品O,商 品P,商 品T,商 品U,商 品V,商 品X,商 品Y,商 品s,商品 A,商品 B,商品 E,商品 F,商品 H,...,商品D,商品E,商品F,商品G,商品H,商品I,商品J,商品K,商品L,商品M,商品N,商品O,商品P,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z,商品a,商品c,商品d,商品e,商品g,商品i,商品j,商品k,商品l,商品o,商品p,商品r,商品s,商品t,商品v,商品x,商品y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,0,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,800,0,0,0,0,1500,1600,2000,0,0,4800,0,1900,0,0,0,0,0,...,6000,6500,7800,7000,10400,13500,16000,16500,19200,19500,21000,27000,20800,25500,36000,28500,28000,12600,41800,27600,24000,20000,0,0,300,0,0,0,0,0,1100,1200,1500,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,0,2400,0,0,0,0,0,0,0,0,0,0,400,0,0,0,0,900,0,0,1400,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7600,5500,6600,7000,12800,8100,11000,14300,13200,16900,23800,25500,30400,30600,32400,32300,40000,37800,39600,43700,31200,20000,2600,0,0,0,0,0,0,1000,0,0,0,0,0,1900,2000,2200,0,0
201903,0,0,500,1300,1600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6000,4000,11400,7700,14400,10800,13000,14300,21600,26000,16800,15000,32000,34000,27000,26600,22000,46200,35200,34500,38400,35000,0,0,0,400,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,2300,0,0,0,0,0,0,0,0,1700,2000,2200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1500,0,0,0,0,0,7500,0,0,0,0,1200,0,...,7200,8000,7800,9100,7200,14400,13000,12100,15600,14300,22400,18000,22400,28900,30600,28500,28000,16800,19800,20700,38400,32500,0,0,0,0,500,700,0,0,0,0,0,0,0,1900,0,0,0,0
201905,0,0,0,0,0,1900,0,0,0,0,0,0,0,100,0,1900,100,0,0,0,0,0,0,0,1100,1300,0,0,0,0,0,2200,0,0,0,0,0,0,600,0,...,5200,9000,9000,14000,12000,13500,8000,15400,12000,20800,23800,24000,28800,15300,32400,20900,26000,33600,8800,36800,31200,45000,0,0,300,0,0,0,0,0,0,1200,0,0,0,0,0,0,0,2500
201906,0,0,0,0,0,0,2300,0,0,0,0,0,0,0,0,0,0,300,0,0,0,700,0,0,0,0,0,1500,0,0,2100,0,0,0,0,100,0,0,0,800,...,7200,6500,9600,7700,8800,12600,20000,15400,14400,19500,15400,19500,24000,23800,27000,34200,22000,29400,33000,25300,24000,30000,0,0,0,0,0,0,900,0,0,0,0,1600,0,0,0,0,2400,0
201907,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1700,0,0,0,0,500,0,0,800,0,0,0,0,1500,0,0,0,2200,0,0,0,0,0,500,0,0,...,6000,4500,9600,11900,12800,13500,17000,9900,20400,13000,18200,25500,38400,18700,25200,38000,34000,23100,52800,32200,38400,27500,0,0,0,0,0,0,900,0,0,0,1500,0,1800,0,0,0,0,0


## データの歪みを考慮し、クレンジングを行う
### 商品名：空白を取り除く、小文字→大文字
### 価格の欠損値：価格変動は無いので同名商品の値段をコピーする
### 顧客名：空白を取り除く

In [16]:
#商品名の修正
uriage_data["item_name"] = uriage_data["item_name"].str.upper() #大文字に合わせる
uriage_data["item_name"] = uriage_data["item_name"].str.replace("  ","")
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ","")

item_name_list = pd.unique(uriage_data["item_name"])
print(len(item_name_list)) #pd.uniqueで指定属性の名前の配列を得ることが可能
print(item_name_list)

26
['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']


In [23]:
#価格の欠損値の補完
#item_priceにある欠損箇所のフラグ
flg_is_null = uriage_data["item_price"].isnull() #bool値をリスト化
#フラグがTrueなら、同じ商品名の値段をコピーする
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"] == trg), "item_price"].max()
    uriage_data["item_price"].loc[(flg_is_null) & (uriage_data["item_name"]==trg)] = price
uriage_data.head()

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [25]:
uriage_data.isnull().sum()

purchase_date     0
item_name         0
item_price        0
customer_name     0
purchase_month    0
dtype: int64

In [29]:
for trg in list(uriage_data["item_name"].sort_values().unique()):
    print(trg + "の最大額：" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].max()) + "と、最小額：" + str(uriage_data.loc[uriage_data["item_name"]==trg]["item_price"].min(skipna=False)))

商品Aの最大額：100.0と、最小額：100.0
商品Bの最大額：200.0と、最小額：200.0
商品Cの最大額：300.0と、最小額：300.0
商品Dの最大額：400.0と、最小額：400.0
商品Eの最大額：500.0と、最小額：500.0
商品Fの最大額：600.0と、最小額：600.0
商品Gの最大額：700.0と、最小額：700.0
商品Hの最大額：800.0と、最小額：800.0
商品Iの最大額：900.0と、最小額：900.0
商品Jの最大額：1000.0と、最小額：1000.0
商品Kの最大額：1100.0と、最小額：1100.0
商品Lの最大額：1200.0と、最小額：1200.0
商品Mの最大額：1300.0と、最小額：1300.0
商品Nの最大額：1400.0と、最小額：1400.0
商品Oの最大額：1500.0と、最小額：1500.0
商品Pの最大額：1600.0と、最小額：1600.0
商品Qの最大額：1700.0と、最小額：1700.0
商品Rの最大額：1800.0と、最小額：1800.0
商品Sの最大額：1900.0と、最小額：1900.0
商品Tの最大額：2000.0と、最小額：2000.0
商品Uの最大額：2100.0と、最小額：2100.0
商品Vの最大額：2200.0と、最小額：2200.0
商品Wの最大額：2300.0と、最小額：2300.0
商品Xの最大額：2400.0と、最小額：2400.0
商品Yの最大額：2500.0と、最小額：2500.0
商品Zの最大額：2600.0と、最小額：2600.0


In [31]:
#顧客名の修正
#商品名の修正
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace("  ","")
kokyaku_data["顧客名"] = kokyaku_data["顧客名"].str.replace(" ","")

print(pd.unique(kokyaku_data["顧客名"]))

['須賀ひとみ' '岡田\u3000敏也' '芳賀希' '荻野愛' '栗田憲一' '梅沢麻緒' '相原ひとり' '新村丈史' '石川まさみ'
 '小栗正義' '大倉晃司' '那須蒼甫' '清水裕次郎' '榊原しぼり' '高沢美咲' '川島友以乃' '唐沢景子' '稲田将也' '秋葉あき'
 '西脇礼子' '内村まさみ' '進藤瞬' '小口豊' '笹原しぼり' '岩佐孝太郎' '河村由樹' '菅原誠治' '榎本薫' '高梨結衣'
 '鶴岡薫' '藤木一恵' '森岡季衣' '田畑正敏' '宇野秀樹' '高原充則' '梅村秀樹' '浅田賢二' '石渡小雁' '吉岡サダヲ'
 '井口寛治' '奥光洋' '佐藤慶二' '東光博' '高田さんま' '合田光' '深田信輔' '島英嗣' '梅津淳子' '日比野徹' '熊倉綾'
 '松田浩正' '楠高史' '岡慶太' '市田寿明' '山西花' '青山鉄洋' '松川綾女' '大滝麗奈' '赤木愛梨' '楠哲平' '堀内聡'
 '八木雅彦' '林勇' '日野夏希' '麻生莉緒' '根岸仁晶' '赤木だん吉' '亀井一徳' '横田遥' '野本仁晶' '西原未華子'
 '五十嵐春樹' '堀江佑' '久保田倫子' '金森なつみ' '城戸芳正' '望月真悠子' '井本マサカズ' '米沢仁晶' '本橋直人'
 '古川信吾' '岩沢那奈' '西島知世' '戸塚美幸' '深井照生' '新谷智花' '有馬徹平' '本多フミヤ' '浜田未華子' '牧田玲那'
 '島孝太郎' '石田花' '星野美嘉' '黒谷長利' '深沢ひろ子' '矢部夏空' '立石茜' '手塚雅之' '松谷愛子' '中田美智子'
 '片瀬長利' '石崎幸子' '明石家明' '小川美菜' '田辺光洋' '神原美嘉' '岩城徹平' '相川良介' '植村遥' '井川真悠子'
 '松元翔太' 'さだ千佳子' '島本研二' '影山輝信' '浅見広司' '石野仁' '小平陽子' '福本美幸' '中仁晶' '田辺きみまろ'
 '熊倉明日' '鈴木一哉' '杉田将也' '徳重優' '松岡ノブヒコ' '藤広之' '谷本愛梨' '松沢育二' '清水佑' '土屋朝陽'
 '鳥居広司' '福島友也' '堀サンタマリア' '井川里穂' '田崎菜々美' '平田鉄二' '井上桃子' '尾形小雁' '沖遥' '矢部美